# Bonus

In [41]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor

In [22]:
transactions_df = pd.read_csv("https://www.dropbox.com/s/360xhh2d9lnaek3/allegro-api-transactions.csv?dl=1")

In [23]:
transactions_df = transactions_df[["main_category", "categories", "it_location", "price"]]
transactions_df.head()

,main_category,categories,it_location,price
0,Komputery,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",Warszawa,59.99
1,"Odzież, Obuwie, Dodatki","['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",Warszawa,4.90
2,Dom i Ogród,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",Leszno,109.90
3,Książki i Komiksy,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",Wola Krzysztoporska,18.50
4,"Odzież, Obuwie, Dodatki","['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",BIAŁYSTOK,19.90


In [28]:
transactions_df["it_location"] = pd.DataFrame(transactions_df["it_location"].str.lower())
transactions_df["it_location"] = pd.DataFrame(transactions_df["categories"].str.lower())
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420020 entries, 0 to 420019
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   main_category  420020 non-null  object 
 1   categories     420020 non-null  object 
 2   it_location    420020 non-null  object 
 3   price          420020 non-null  float64
dtypes: float64(1), object(3)
memory usage: 12.8+ MB


## Train-test split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(transactions_df.drop("price", axis=1), 
                                                    transactions_df["price"], 
                                                    test_size=0.25, shuffle=True, random_state=42)

## Modeling
### Target encoding
Poniższe informacje wzięte są z [https://maxhalford.github.io/blog/target-encoding/](https://maxhalford.github.io/blog/target-encoding/) .


Użyjemy tutaj target encodingu z parametrem `smoothing`, dzięki czemu do kodowania zamiast zwykłej średniej targetu po grupach wykorzystana zostanie wartość $$\frac{n \times \bar{x} + m \times w}{n + m},$$
gdzie
- $n$ to liczność danej grupy,
- $\bar{x}$ to średnia targetu z danej grupy,
- $w$ to ogólna średnia targetu,
- $m$ to wartość `smoothing`($m \in [0, \infty)$).

Taki encoding jest użyteczny, np. gdy wiele grup jest nielicznych. Wtedy sama średnia może z dużym błędem estymować wartość oczekiwaną targetu w tych grupach, więc pomocne może być uwzględnienie średniej ogólnej. Wartość `smoothing` określa jak bardzo polegamy na ogólnej średniej targetu.

### Regularyzacja
Polega na wprowadzeniu dodatkowego bias'u(błędu na danych treningowych), aby zmniejszyć variance(błąd na danych testowych). Jedne z najbardziej znanych technik to 
- `l1`(Lasso), kara w postaci kwadratu 2 normy wektora wag(z regresji liniowej),  
- `l2`(Ridge), kara w postaci 1 normy wektora wag.

Dane techniki użyteczne są np. przy obecności skorelowanych zmiennych lub mało istotnych cech.

In [34]:
def get_scores(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return (r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred, squared=False))


def modeling(X_train, y_train, reg):

    if reg == "lasso":
        clf = Lasso()
    elif reg == "ridge":
        clf = Ridge()
    elif reg == "least_sq":
        clf = LinearRegression()
    else:
        raise NameError("reg: " + reg + " is unknown")

    pipeline =  Pipeline([("encoder", TargetEncoder(smoothing=300)), ("clf", clf)])

    return pipeline.fit(X_train, y_train)


def experiment(X_train, X_test, y_train, y_test):
    reg = ["least_sq", "lasso", "ridge"]
    
    res_merged = None
    for i in range(3):
        model = modeling(X_train, y_train, reg[i])
        scores = get_scores(model, X_test, y_test)
        res = pd.DataFrame.from_dict({"model": [reg[i]],
                                     "r2": [scores[0]],
                                     "rmse": [scores[1]]})
        res_merged = pd.concat([res_merged, res], axis=0).reset_index(drop=True)
        
    return res_merged

In [42]:
result = experiment(X_train, X_test, y_train, y_test)

pipeline = Pipeline([("encoder", TargetEncoder(smoothing=300)), 
                     ("clf", GradientBoostingRegressor(n_jobs=-1))])
pipeline.fit(X_train, y_train)

scores = get_scores(pipeline, X_test, y_test)
pd.concat([result,
          pd.DataFrame.from_dict({"model": ["gbc"],
                                  "r2": [scores[0]],
                                  "rmse": [scores[1]]})], 
          axis=0).reset_index(drop=True)

,model,r2,rmse
0,least_sq,0.064692,476.390912
1,lasso,0.064695,476.390246
2,ridge,0.064692,476.390912
3,gbc,0.101214,466.997242
